In [ ]:
#import pathlib
from pathlib import Path
from typing import List, Tuple, Union
import csv
#import copy
import numpy as np
#import matplotlib.pyplot as plt
import serpentTools
from serpentTools.parsers.results import ResultsReader
#import click
#import matplotlib.pyplot as plt

serpentTools.settings.rc["serpentVersion"] = "2.1.32"
P = 48 #max no of FA
Z = 6  #max no of slices

AV = 0.6221408E+24

# get atomic wt numbers, store in atomic_wt_dict
with open("isotope_awt_list.csv", mode='r') as infile:
    reader = csv.reader(infile)
    with open('coors_new.csv', mode='w') as outfile:
        writer = csv.writer(outfile)
        atomic_wt_dict = {rows[0]:rows[1] for rows in reader}


def discharge_bu() -> None:
    res_list = []

    with serpentTools.settings.rc:
        serpentTools.settings.rc["verbosity"] = "error"
    dep = serpentTools.read("wh_lfr_dep.m")
    
    return dep

dep = discharge_bu()


def split_name(name: str)->Tuple[int, int]:
    """The functions take a fuel name, separates the P and Z and takes out the indexes
        Note: It s made only for this specific nomeclature, needs modification if you 
        want more fancy names

    Args:
        name (str): Takes in a standard fuel position name "fuelPxxxZxxx"

    Returns:
        Tuple[int, int]: Gives out a tuple with the val of P and Z
    """
    without_fuel = name[4:]
    split_without_fuel  = without_fuel.split("Z")
    p_num = int(split_without_fuel[0][1:])
    z_num = int(split_without_fuel[1])
    return p_num, z_num

### NEW VERSION

#### This is the part to be used inside the run to get the right _dep.m file

set inventory			% - Sets inventory for dep.m file 

Th-227 Th-228 Th-229 Th-230 Th-231 Th-232 Th-233 Th-234
Pa-231 Pa-232 Pa-233
U-232  U-233  U-234  U-235  U-236  U-237  U-238  U-239  U-240  U-241 
Np-235 Np-236 Np-237 Np-238 Np-239
Pu-236 Pu-237 Pu-238 Pu-239 Pu-240 Pu-241 Pu-242 Pu-243 Pu-244
Am-241 Am-242 Am-242m Am-243 Am-244 Am-244m 
Cm-240 Cm-241 Cm-242 Cm-243 Cm-244 Cm-245 Cm-246 Cm-247 Cm-248 Cm-249 Cm-250

Th Pa U Np Pu Am Cm			% - total list

In [ ]:
materials = dep.materials.keys()
#create my own dict with fuel vol's
fuel_vol_dict = {}

# Go through each material index
for fuel_vol in materials:
    if fuel_vol == "total":
        # The continue, skips the loop interation at that point
        continue
    # Add the materials until 52 pos, exclude the total ones, big total and lost (maybe there is some way more intuitive)
    # Last column (last BU step in sss2) is taken and summed up for each fuel volume, store in dic wth key mdens_sum
    fuel_vol_dict[fuel_vol] = {"mdens_sum": dep.materials[fuel_vol].mdens[:, -1][0:52].sum(), "serpent_burnup": dep.materials[fuel_vol].burnup[-1]}

for fuel_vol in fuel_vol_dict:
    p_index, z_index = split_name(str(fuel_vol))
    mdens_to_divide_key = f"fuelP1Z{z_index}"
    mdens_to_divide = dep.materials[mdens_to_divide_key].mdens[:, -1][0:52].sum()
    ratio = fuel_vol_dict[fuel_vol]["mdens_sum"] / mdens_to_divide
    fuel_vol_dict[fuel_vol]["mdens_ratio"] = ratio
    #print(ratio)

for fuel_vol in fuel_vol_dict.keys():
    p_index, z_index = split_name(fuel_vol)
    burn_up = fuel_vol_dict[fuel_vol]["serpent_burnup"]
    #print(burn_up)
    if p_index == 1:
        fuel_vol_dict[fuel_vol]["corrected_burnup"] = burn_up
        continue
    
    material_name = f"fuelP{p_index - 1}Z{z_index}"
    ratio = fuel_vol_dict[material_name]["mdens_ratio"]
    corrected_burnup = burn_up * ratio
    fuel_vol_dict[fuel_vol]["corrected_burnup"] = corrected_burnup

average_burnups = {}
for fuel_vol in fuel_vol_dict.keys():
    p_index, _ = split_name(fuel_vol)
    average_burnup = 0
    for zidx in range(1 , Z + 1):
        key_construct = f"fuelP{p_index}Z{zidx}"
        average_burnup += fuel_vol_dict[key_construct]["corrected_burnup"] / Z
    average_burnups[f"P{p_index}"] = average_burnup

average_burnups_axial = {}
for fuel_vol in fuel_vol_dict.keys():
    _, z_index = split_name(fuel_vol)
    average_axial_burnup = 0
    for pidx in range(1 , P + 1):
        key_construct = f"fuelP{pidx}Z{z_index}"
        average_axial_burnup += fuel_vol_dict[key_construct]["corrected_burnup"]
    average_burnups_axial[f"Z{z_index}"] = average_axial_burnup


In [ ]:
average_burnups_axial

In [ ]:
sum([value for key, value in average_burnups.items()])


In [ ]:
sum([value for key, value in average_burnups_axial.items()]) / Z

In [ ]:
average_burnups

In [ ]:
dep = serpentTools.read("wh_lfr_dep.m")

In [ ]:
dir(dep.materials["total"])


In [ ]:
dayPoints = totalfuel.days
iso = ['plutonium', 'U238']
zai = [94, 922380]

totalfuel = dep.materials["total"]

#totalfuel.data["mass"]

totalfuel.getValues('days', 'mass', totalfuel.days, iso)
#totalfuel.names

In [ ]:
totalfuel.days

In [ ]:
import json

x = dep.materials["fuelP48Z1"].toDataFrame("a", names=[json.dumps(totalfuel.names)], time="days")

In [ ]:
totalfuel.getValues('days', 'a', dayPoints, iso)